In [1]:
from qcrbox_wrapper import *

In [2]:
qcrbox = QCrBoxWrapper('localhost', 11000)

In [3]:
qcrbox.application_dict

{'XHARPy-GPAW': XHARPy-GPAW()}

In [4]:
xharpy = qcrbox.application_dict['XHARPy-GPAW']

In [5]:
help(xharpy)

Help on QCrBoxApplication in module qcrbox_wrapper:

XHARPy-GPAW()
    Represents the XHARPy-GPAW application (v. 0.2.0) in QCrBox
    
    Methods:
        atom_form_fact_gpaw(
            input_cif_path: str,
            output_tsc_path: str,
            functional: str,
            gridspacing: str
        )
    
        ha_refine(
            input_cif_path: str,
            output_cif_path: str,
            functional: str,
            gridspacing: str
        )



In [6]:
calc = xharpy.atom_form_fact_gpaw(
    input_cif_path='/mnt/qcrbox/shared_files/test_xharpy_tsc/input_epox.cif',
    output_tsc_path='/mnt/qcrbox/shared_files/test_xharpy_tsc/epoxide.tsc',
    functional='LDA',
    gridspacing=0.24
)

In [7]:
calc.wait_while_running(1.0)

In [8]:
calc2 = xharpy.ha_refine(
    input_cif_path='/mnt/qcrbox/shared_files/test_xharpy_refine/input_epox.cif',
    output_cif_path='/mnt/qcrbox/shared_files/test_xharpy_refine/epoxide_out.cif',
    functional='PBE',
    gridspacing=0.20
)
print(calc2.status)
calc2.wait_while_running(1.0)
print(calc2.status)

QCrBoxCalculationStatus(calculation_id=2, command_id=2, started_at='2024-02-26T14:36:23.050190', status='unknown', status_details={'status': 'unknown', 'details': {'msg': "An error occurred when trying to determine the status of the future: '_diffrn_refln.index_h'"}})
QCrBoxCalculationStatus(calculation_id=2, command_id=2, started_at='2024-02-26T14:36:23.050190', status='unknown', status_details={'status': 'unknown', 'details': {'msg': "An error occurred when trying to determine the status of the future: '_diffrn_refln.index_h'"}})
